Made on google colab

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
!python -V

Python 3.10.12


In [3]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.17.1'

In [4]:
!wget http://bit.ly/mlbookcamp-pants -O pants.jpg

--2024-12-16 20:56:46--  http://bit.ly/mlbookcamp-pants
Resolving bit.ly (bit.ly)... 67.199.248.10, 67.199.248.11
Connecting to bit.ly (bit.ly)|67.199.248.10|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/alexeygrigorev/clothing-dataset-small/master/test/pants/4aabd82c-82e1-4181-a84d-d0c6e550d26d.jpg [following]
--2024-12-16 20:56:46--  https://raw.githubusercontent.com/alexeygrigorev/clothing-dataset-small/master/test/pants/4aabd82c-82e1-4181-a84d-d0c6e550d26d.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23048 (23K) [image/jpeg]
Saving to: ‘pants.jpg’

pants.jpg           100%[===================>]  22.51K  --.-KB/s    in 0.002s  

2024-12-16 20:56:46 (13.4 MB/s) - ‘pants.jp

In [5]:
from google.colab import files
uploaded = files.upload()

Saving clothing_model_from_module8.keras to clothing_model_from_module8.keras


In [6]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

In [7]:
model = keras.models.load_model('clothing_model_from_module8.keras')

In [8]:
img = load_img('pants.jpg', target_size=(299, 299))

x = np.array(img)
X = np.array([x])

X = preprocess_input(X)

In [9]:
preds = model.predict(X)


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step


In [10]:
preds

array([[-2.229012 , -4.819431 , -1.3372766, -1.046778 ,  7.787136 ,
        -0.3799843, -2.9674664,  2.9388351, -2.46029  , -3.8072712]],
      dtype=float32)

In [11]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [12]:
dict(zip(classes, preds[0]))


{'dress': -2.229012,
 'hat': -4.819431,
 'longsleeve': -1.3372766,
 'outwear': -1.046778,
 'pants': 7.787136,
 'shirt': -0.3799843,
 'shoes': -2.9674664,
 'shorts': 2.9388351,
 'skirt': -2.46029,
 't-shirt': -3.8072712}

Keras to TF Lite

In [13]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('clothing-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

Saved artifact at '/tmp/tmpn5eyxlog'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name='input_layer_4')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  131985977951328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131985977961712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131985977963824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131985977957136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131985977959424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131985977963472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131985978098432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131985978102832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131985978099488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131985978102128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131985978105

In [14]:
import tensorflow.lite as tflite


In [15]:
interpreter = tflite.Interpreter(model_path='clothing-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [16]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [17]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

{'dress': -2.229012,
 'hat': -4.819433,
 'longsleeve': -1.3372766,
 'outwear': -1.0467798,
 'pants': 7.7871327,
 'shirt': -0.3799859,
 'shoes': -2.9674675,
 'shorts': 2.9388394,
 'skirt': -2.460288,
 't-shirt': -3.8072734}

Remove TF Dependency

In [18]:
from PIL import Image


In [19]:
with Image.open('pants.jpg') as img:
    img = img.resize((299, 299), Image.NEAREST)

In [20]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [21]:
x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [22]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [23]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

{'dress': -2.229012,
 'hat': -4.819433,
 'longsleeve': -1.3372766,
 'outwear': -1.0467798,
 'pants': 7.7871327,
 'shirt': -0.3799859,
 'shoes': -2.9674675,
 'shorts': 2.9388394,
 'skirt': -2.460288,
 't-shirt': -3.8072734}

Simpler way of doing it

In [24]:
!pip install keras-image-helper


In [25]:
!pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime


Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.1 MB/s eta 0:00:00


In [26]:
#import tensorflow.lite as tflite
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

In [27]:
interpreter = tflite.Interpreter(model_path='clothing-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [28]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))


In [29]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [30]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [31]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

{'dress': -2.2290115,
 'hat': -4.8194337,
 'longsleeve': -1.3372767,
 'outwear': -1.0467795,
 'pants': 7.787133,
 'shirt': -0.379986,
 'shoes': -2.9674675,
 'shorts': 2.9388397,
 'skirt': -2.4602888,
 't-shirt': -3.807273}